In [1]:
!pip install emoji

In [1]:
import numpy as np
import pandas as pd
import emoji

In [2]:
emoji_dictionary = {"0": "\u2764\uFE0F",
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [3]:
emoji.emojize(":baseball:")

'⚾'

In [4]:
for label in emoji_dictionary:
    print(emoji.emojize(emoji_dictionary[label]))

❤️
⚾
😁
😓
🍴


In [5]:
train = pd.read_csv('dataset/train_emoji.csv',header=None)
test = pd.read_csv('dataset/test_emoji.csv',header=None)

In [6]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [7]:
for i in range(30):
    print(train.iloc[i,0],'--->',emoji.emojize(emoji_dictionary[str(train.iloc[i,1])]))

never talk to me again ---> 😓
I am proud of your achievements ---> 😁
It is the worst day in my life ---> 😓
Miss you so much ---> ❤️
food is life ---> 🍴
I love you mum ---> ❤️
Stop saying bullshit ---> 😓
congratulations on your acceptance ---> 😁
The assignment is too long  ---> 😓
I want to go play ---> ⚾
she did not answer my text  ---> 😓
Your stupidity has no limit ---> 😓
how many points did he score ---> ⚾
my algorithm performs poorly ---> 😓
I got approved ---> 😁
Stop shouting at me ---> 😓
Sounds like a fun plan ha ha ---> 😁
no one likes him ---> 😓
the game just finished ---> ⚾
I will celebrate soon ---> 😁
So sad you are not coming ---> 😓
She is my dearest love ---> ❤️
Good job ---> 😁
It was funny lol ---> 😁
candy is life  ---> 😁
The chicago cubs won again ---> ⚾
I am hungry ---> 🍴
I am so excited to see you after so long ---> 😁
you did well on you exam ---> 😁
lets brunch some day ---> 🍴


In [8]:
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:],dtype='f4')
        embeddings[word] = vector

In [9]:
print(embeddings['the'])

[ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01]


In [10]:
from keras.utils import to_categorical

X_train = train[0]
X_test = test[0]

y_train = to_categorical(train[1])
y_test = to_categorical(test[1])

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [11]:
def getEmbeddings(X):
    embeddingData = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        words = X.iloc[ix].lower().split()
        for jx in range(len(words)):
            embeddingData[ix,jx,:] = embeddings[words[jx]]
    return embeddingData

In [12]:
emb_XT = getEmbeddings(X_train)

In [18]:
print(X_train[0])
# print(embeddings['never'])
# print(emb_XT[0][0])
print(emb_XT.shape)

never talk to me again
(132, 10, 50)


In [19]:
emb_Xt = getEmbeddings(X_test)

In [20]:
print(emb_Xt.shape)

(56, 10, 50)


In [21]:
from keras.layers import Dense,LSTM,Dropout
from keras.models import Sequential
from keras import Input

In [22]:
model = Sequential()
model.add(Input(shape=(10,50)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(16))
model.add(Dropout(0.3))
model.add(Dense(5,activation='softmax'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 10, 32)              │          10,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,845 (54.08 KB)

 Trainable params: 13,845 (54.08 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
model.fit(emb_XT,y_train,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Epoch 1/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 143ms/step - accuracy: 0.1821 - loss: 1.6243 - val_accuracy: 0.0714 - val_loss: 1.6309
Epoch 2/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2679 - loss: 1.5939 - val_accuracy: 0.1429 - val_loss: 1.6265
Epoch 3/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.3921 - loss: 1.5457 - val_accuracy: 0.1429 - val_loss: 1.6311
Epoch 4/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.3752 - loss: 1.5214 - val_accuracy: 0.1429 - val_loss: 1.6380
Epoch 5/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3614 - loss: 1.5041 - val_accuracy: 0.2143 - val_loss: 1.6441
Epoch 6/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4067 - loss: 1.4784 - val_accuracy: 0.1429 - val_loss: 1.6506
Epoch 7/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3335 - loss: 1.4627 - val_accuracy: 0.3571 - val_loss: 1.6372
Epoch 8/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4255 - loss: 1.4599 - val_accuracy: 0.3571 - val_loss: 1.6091

In [24]:
model.evaluate(emb_Xt,y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5699 - loss: 1.3923


[1.3200280666351318, 0.5892857313156128]

In [25]:
ypred = model.predict(emb_Xt)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step


In [26]:
for i in range(30):
    print(X_test.iloc[i])
    print(emoji.emojize(emoji_dictionary[str(np.argmax(y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(ypred[i]))]))

I want to eat	
🍴
🍴
he did not answer	
😓
😓
he got a raise	
😁
😓
she got me a present	
❤️
😁
ha ha ha it was so funny	
😁
😁
he is a good friend	
❤️
😁
I am upset	
❤️
😓
We had such a lovely dinner tonight	
❤️
😁
where is the food	
🍴
🍴
Stop making this joke ha ha ha	
😁
😁
where is the ball	
⚾
⚾
work is hard	
😓
😁
This girl is messing with me	
😓
❤️
are you serious ha ha	
😁
😓
Let us go play baseball	
⚾
⚾
This stupid grader is not working 	
😓
😓
work is horrible	
😓
😁
Congratulation for having a baby	
😁
😁
stop messing around	
😓
🍴
any suggestions for dinner	
🍴
🍴
I love taking breaks	
❤️
❤️
you brighten my day	
😁
❤️
I boiled rice	
🍴
🍴
she is a bully	
😓
😁
Why are you feeling bad	
😓
😓
I am upset	
😓
😓
I worked during my birthday	
😓
😁
My grandmother is the love of my life	
❤️
❤️
enjoy your break	
😁
🍴
valentine day is near	
❤️
😁
